In [1]:
#! pip install pyoso

In [1]:
from dotenv import load_dotenv
import os
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [2]:
df_pkg = client.to_pandas("""
SELECT DISTINCT 
  package_artifact_source,
  package_artifact_name
FROM package_owners_v0 
WHERE package_owner_artifact_namespace = 'libp2p'
""")
df_pkg

,package_artifact_source,package_artifact_name
0,NPM,@libp2p/interface-pubsub
1,NPM,@libp2p/interface-peer-id
2,GO,github.com/libp2p/go-libp2p-peerstore
3,GO,github.com/libp2p/go-cidranger
4,RUST,libp2p-identify
...,...,...
253,NPM,interface-connection
254,GO,github.com/libp2p/go-libp2p-consensus
255,GO,github.com/libp2p/go-libp2p-nat
256,GO,github.com/libp2p/go-yamux/v3


In [11]:
df_deps = client.to_pandas("""
SELECT DISTINCT
  CONCAT(s.from_artifact_namespace, '/', s.from_artifact_name) AS dependent,
  COALESCE(r.star_count,0) AS stars,
  s.to_package_artifact_name AS package,
  s.to_package_artifact_source AS source
FROM sboms_v0 AS s
JOIN package_owners_v0 AS pkgs ON
  s.to_package_artifact_source = pkgs.package_artifact_source
  AND s.to_package_artifact_name = pkgs.package_artifact_name
JOIN repositories_v0 AS r
  ON s.from_artifact_id = r.artifact_id
WHERE package_owner_artifact_namespace = 'libp2p'
ORDER BY 2
""")
df_deps.tail()

,dependent,stars,package,source
26317,ipfs/kubo,16461,github.com/libp2p/go-libp2p-peerstore,GO
26318,ipfs/kubo,16461,github.com/libp2p/go-reuseport,GO
26319,ipfs/kubo,16461,github.com/libp2p/go-libp2p-http,GO
26320,ipfs/kubo,16461,github.com/libp2p/go-libp2p-core,GO
26321,ipfs/kubo,16461,github.com/libp2p/go-nat,GO


In [12]:
df_deps['dependent'].nunique()

2920

In [9]:
df_deps.groupby(['package', 'source'])['dependent'].nunique().sort_values(ascending=False).head(30)

package                               source
libp2p-crypto                         NPM       1632
peer-id                               NPM       1629
peer-info                             NPM       1023
libp2p-crypto-secp256k1               NPM       1002
github.com/libp2p/go-buffer-pool      GO         577
libp2p                                NPM        470
github.com/libp2p/go-libp2p           GO         465
@libp2p/logger                        NPM        455
@libp2p/peer-id                       NPM        436
github.com/libp2p/go-flow-metrics     GO         354
github.com/libp2p/go-msgio            GO         346
@libp2p/interface                     NPM        322
github.com/libp2p/go-netroute         GO         308
@libp2p/interfaces                    NPM        301
@libp2p/crypto                        NPM        295
github.com/libp2p/go-libp2p-asn-util  GO         291
github.com/libp2p/go-nat              GO         289
github.com/libp2p/go-reuseport        GO         288
l